In [ ]:
# This block enables local development of the scripts in the `src` folder while using the functions directly in this notebook without restart
%load_ext autoreload
%autoreload 2

# Modules

## Python

In [ ]:
import sys
import os
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
import pdb
import matplotlib.pyplot as plt
import seaborn as sns

## Package

In [ ]:
sys.path.append(r"C:\Users\SachaGobeyn\GitHub\CNWS-processing-scripts\cnws\src\cfactor")
from cfactor import cfactor,init
from general import initialize_files,prepare_parcel_input

# Trailrun Maarkebeek

## Gewasgegevens

In [ ]:
#path_inputdata = r"C:\Users\SachaGobeyn\Dropbox (Fluves)\ALBON\C-factor\2. ANALYSES\-testtutorial\inputdata"
path_sources = os.path.join("..","..","src","cfactor","data","raw")
path_tempdata = os.path.join("..","..","src","cfactor","temp")
path_resmap  = os.path.join(r"C:\Users\SachaGobeyn\Dropbox (Fluves)\ALBON\C-factor\2. ANALYSES\analyse_Vlaanderen\Maarkedal")

paths_teeltdata= initialize_files(path_sources)

## Percelenkaart

In [ ]:
in_Shp = {}
in_Shp["2015"] = Path(r"C:\Users\SachaGobeyn\Dropbox (Fluves)\ALBON\DATA\GIS-data\thematic\Landbouwgebruikspercelen\LandbouwVisserij\Landbouwgebruikspercelen_2015_(toestand_10-08-2016)_la72.shp")
in_Shp["2016"] = Path(r"C:\Users\SachaGobeyn\Dropbox (Fluves)\ALBON\DATA\GIS-data\thematic\Landbouwgebruikspercelen\LandbouwVisserij\Landbouwgebruikspercelen_2016_(toestand_16-01-2017)_la72.shp")
clip_Shp = Path(r"C:\Users\SachaGobeyn\Dropbox (Fluves)\ALBON\C-factor\2. ANALYSES\analyse_Vlaanderen\Maarkedal\catchment_shape\Maarkebeek.shp")

parcel_list = prepare_parcel_input(in_Shp,clip_Shp,path_tempdata)

## Run cfactor model for Maarkebeek

In [ ]:
parcel, crop_prop, ggg, grid = init(parcel_list,paths_teeltdata,2016)
out = cfactor(parcel, grid, ggg, crop_prop, 2016,
                                        output_map=os.path.join(path_resmap),ffull_output=True, 
                                        multiprocessing=True)

# Volledig Vlaanderen

In [ ]:
catchments = ["Demerbekken"]
             #   ["Bekken_Brugse_polders","Bekken_Gentse_kanalen","Beneden-Scheldebekken",
             # "Boven-scheldebekken","Demerbekken","Dijlebekken","Ijzerbekken","Leiebekken",
             # "Maasbekken_N","Maasbekken_E","Netebekken","Voeren"]
catchMap =  r"C:\Users\SachaGobeyn\Dropbox (Fluves)\ALBON\Modelberekeningen\v8\Vlaanderen_Bekkens\catchments"
resMap = r"C:/Users/SachaGobeyn/Dropbox (Fluves)/ALBON/C-factor/2. ANALYSES/analyse_Vlaanderen/"

## Prepare parcels

In [ ]:
in_Shp = {}
in_Shp["2015"] = Path(r"C:\Users\SachaGobeyn\Dropbox (Fluves)\ALBON\DATA\GIS-data\thematic\Landbouwgebruikspercelen\LandbouwVisserij\Landbouwgebruikspercelen_2015_(toestand_10-08-2016)_la72.shp")
in_Shp["2016"] = Path(r"C:\Users\SachaGobeyn\Dropbox (Fluves)\ALBON\DATA\GIS-data\thematic\Landbouwgebruikspercelen\LandbouwVisserij\Landbouwgebruikspercelen_2016_(toestand_16-01-2017)_la72.shp")

for i in catchments:
    print(i)
    clip_Shp = Path(os.path.join(catchMap,"%s.shp"%i))
    parcel_list = prepare_parcel_input(in_Shp,clip_Shp,path_tempdata)
    parcel_list = pd.read_csv(os.path.join(resMap,"parcel_list_%s.csv"%i))
    parcel, crop_prop, ggg, grid = init(parcel_list,paths_teeltdata,2016)
    parcel.to_csv(os.path.join(resMap,"parcel_list_%s.shp"%i))
    #parcel, crop_prop, ggg, grid = init(parcel_list,paths_teeltdata,2016)
    #out = cfactor(parcel, grid, ggg, crop_prop, 2016,
    #                                        output_map=os.path.join(path_resmap),ffull_output=True, 
    #                                        multiprocessing=True)

## Run cfactor

In [ ]:
for i in catchments:
    parcel =  pd.read_csv(os.path.join(resMap,"parcel_list_%s.shp"%i))
    out = cfactor(parcel, grid, ggg, crop_prop, 2016,
                                            output_map=os.path.join(path_resmap),ffull_output=False, 
                                            multiprocessing=True)
    out.to_csv(os.path.join(resMap,"cfactor_%s.csv"%i))

## Analyse

### load

In [ ]:
cfactor = []
parcellists = []

catchments_ = ["Demerbekken"]
for i in catchments_:
    parcel_list = pd.read_csv(os.path.join(resMap,"parcel_list_%s.csv"%i))
    parcel_list["catchm"] = i
    parcellists.append(parcel_list)
    parcel_list = parcel_list[parcel_list["jaar"]==2016]
    gewas = parcel_list[["perceel_id","GWSCOD","catchm"]][parcel_list["type"]==2]
    parcel_list = parcel_list.groupby(["perceel_id","REF_ID","catchm"]).aggregate({"type":np.sum}).reset_index()
    parcel_list = parcel_list[parcel_list["type"]<7]
    temp = pd.read_csv(os.path.join(resMap,"cfactor_%s.csv"%i))
    temp = temp.merge(parcel_list,on="perceel_id",how="left").merge(gewas,on="perceel_id",how="left")
    c.append(temp)
    
parcellists = pd.concat(parcellists)
c = pd.concat(c)

types = {2:"H",3:"V+H",5:"H+N",6:"V+H+N"}

c = c.replace(types)

### boxplot

In [ ]:
for i in gewas["GWSCOD"].unique()[0:10]:
    fig,ax = plt.subplots()
    ax =  sns.boxplot(x="GWSCOD", y="C", data=c[c["GWSCOD"]==i],hue="type")

In [ ]:
plt.hist(cfactor["type"])

In [ ]:
parcellists[parcellists["perceel_id"]==74865]

In [ ]:
temp = gpd.read_file(r"C:\Users\SachaGobeyn\Dropbox (Fluves)\ALBON\DATA\GIS-data\thematic\Landbouwgebruikspercelen\LandbouwVisserij\Landbouwgebruikspercelen_2016_(toestand_16-01-2017)_la72.shp")

In [ ]:
cfactor